# import libraries:

In [ ]:
import numpy as np
import sep
# additional setup for reading the test image and displaying plots
from astropy.io import fits
#import fitsio
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.patches import Ellipse
%matplotlib inline

rcParams['figure.figsize'] = [10., 8.]

# Following the tutorial from https://sep.readthedocs.io/en/v1.0.x/tutorial.html

# Reading in the image as well as displaying it(Note I am using an image provided by the professor rather then the tutorial image. Image is called HorseHead.fits and can be located in the files section on canvas):

In [ ]:
fname = "HorseHead.fits"
sdata = fits.getdata(fname)
sdata = sdata.astype(np.float32)
m, s = np.mean(sdata), np.std(sdata)
plt.imshow(sdata, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar();
plt.savefig('tutorialoriginalimage.png')

# Process of removing the background

In [ ]:
sbkg = sep.Background(sdata) 

# getting the global mean of the background:

In [ ]:
print(sbkg.globalback)
print(sbkg.globalrms)

# stores background into 2d array and prints it: 

In [ ]:
sbkg_image = sbkg.back()
plt.imshow(sbkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig('tutorialbackgroundimage.png')

# getting the background noise: 

In [ ]:
sbkg_rms = sbkg.rms()
plt.imshow(sbkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig('tutorialnoiseimage.png')

# subtracting the background noise from 

In [ ]:
sdata_sub = sdata - sbkg

# start trying to detect objects

In [ ]:
sobjects = sep.extract(sdata_sub, 1.5, err=sbkg.globalrms)
#Now we will print out the number of objects detected
len(sobjects)

# Now we will circle all of the objects which were detected

In [ ]:
# plot background-subtracted image
sfig, sax = plt.subplots()
sm, ss = np.mean(sdata_sub), np.std(sdata_sub)
sim = sax.imshow(sdata_sub, interpolation='nearest', cmap='gray',
               vmin=sm-ss, vmax=sm+ss, origin='lower')

# plot an ellipse for each object
for i in range(len(sobjects)):
    se = Ellipse(xy=(sobjects['x'][i], sobjects['y'][i]),
                width=6*sobjects['a'][i],
                height=6*sobjects['b'][i],
                angle=sobjects['theta'][i] * 180. / np.pi)
    se.set_facecolor('none')
    se.set_edgecolor('red')
    sax.add_artist(se)
plt.savefig('tutorialcircledimage.png')

# Additional logic: performing a simple circular aperture photometry and printing the results

In [ ]:
flux, fluxerr, flag = sep.sum_circle(sdata_sub, sobjects['x'], sobjects['y'], 3.0, err=sbkg.globalrms, gain=1.0)
# show the first 10 objects results:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))

# repeating tutorial below with new the hubble image instead

# Reading in the image as well as displaying it

In [ ]:
fname = "hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits"
data = fits.getdata(fname)
data = data.astype(np.float32) #need to reduce the size of the data or else it will cause an error 
m, s = np.mean(data), np.std(data)
plt.imshow(data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower') #showing the image
plt.colorbar();
plt.savefig('originalimage.png')

# Process of removing the background

In [ ]:
bkg = sep.Background(data) 

# getting the global mean of the background:

In [ ]:
print(bkg.globalback)
print(bkg.globalrms)

# stores background into 2d array and prints it: 

In [ ]:
bkg_image = bkg.back()
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig('backgroundimage.png')

# getting the background noise:

In [ ]:
bkg_rms = bkg.rms()
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig('noiseimage.png')

# subtracting the background noise from

In [ ]:
data_sub = data - bkg

# start trying to detect objects

In [ ]:
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms) #threshold = 1.5
#Now we will print out the number of objects detected
print("The number of sources found: ", len(objects))
print(type(objects))

# Now we will circle all of the objects which were detected

In [ ]:
# plot background-subtracted image
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray',
               vmin=m-s, vmax=m+s, origin='lower')

# plot an ellipse for each object
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)
plt.savefig('circledimage.png')

# Make a histogram of the fluxes: 

In [ ]:
#getting the flux data to start off 
new = objects['flux']
#log the flux result because some points are really big
new = np.log10(new)

In [ ]:
histogram = plt.hist(new, bins='auto')
plt.xlabel('value')
plt.ylabel('occurances')
plt.title('Histogram of fluxes')

# Make a histogram of the AB magnitude: 

In [ ]:
#Following formula from stci site m_ab = -2.5*log10(flux) - 48.60 
m_ab = -2.5 * np.log10(objects['flux']) - 48.60
histogram = plt.hist(m_ab, bins='auto')
plt.xlabel('value')
plt.ylabel('occurances')
plt.title('Histogram of ab magnitude')